In [1]:
import pandas as pd
import numpy as np

In [2]:
feats = np.load('aggregated_feats.npy')

In [4]:
clusters = pd.read_csv('cluster_data.csv')

In [5]:
clusters.head()

,lat,lon,cluster_cons,cluster_weight,cluster_hh_f34,cluster_hh_f35,cluster_hh_f34_na,cluster_hh_f35_na,cluster_count,nightlights
0,-17.095150,35.217213,13.004266,189.1064,0.8750,4231.250000,0.0,0.5000,16,0.000000
1,-17.092351,35.114643,10.251386,165.9555,0.1250,1140.000000,0.0,0.8750,16,0.000000
2,-17.016698,35.079629,14.135409,287.5741,0.1250,750.000000,0.0,0.8750,16,0.000000
3,-16.977243,35.205706,12.742217,223.7515,0.1875,1833.333333,0.0,0.8125,16,0.121212
4,-16.956385,35.168967,7.689422,172.6025,0.2500,750.000000,0.0,0.7500,16,0.502674


In [6]:
y = clusters['cluster_cons'].values
y_log = np.log(y)

In [7]:
# This is a bunch of code from the Jean et al Github that is modified to work with Python3 and our data

import numpy as np
import pandas as pd
import random
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import sklearn.linear_model as linear_model
import matplotlib.pyplot as plt
from matplotlib.collections import EllipseCollection
import seaborn as sns


def predict_consumption(
    X, y, k=5, k_inner=5, points=10,
        alpha_low=1, alpha_high=5, margin=0.25):
    """
    Plots predicted consumption
    """
    y_hat, r2 = run_cv(X, y, k, k_inner, points, alpha_low, alpha_high)
    return X, y, y_hat, r2


def run_cv(X, y, k, k_inner, points, alpha_low, alpha_high, randomize=False):
    """
    Runs nested cross-validation to make predictions and compute r-squared.
    """
    alphas = np.logspace(alpha_low, alpha_high, points)
    r2s = np.zeros((k,))
    y_hat = np.zeros_like(y)
    kf = KFold(n_splits=k, shuffle=True)
    fold = 0
    for train_idx, test_idx in kf.split(X):
        r2s, y_hat, fold = evaluate_fold(
            X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
            randomize)
    return y_hat, r2s.mean()


def scale_features(X_train, X_test):
    """
    Scales features using StandardScaler.
    """
    X_scaler = StandardScaler(with_mean=True, with_std=False)
    X_train = X_scaler.fit_transform(X_train)
    X_test = X_scaler.transform(X_test)
    return X_train, X_test


def train_and_predict_ridge(alpha, X_train, y_train, X_test):
    """
    Trains ridge model and predicts test set.
    """
    ridge = linear_model.Ridge(alpha)
    ridge.fit(X_train, y_train)
    y_hat = ridge.predict(X_test)
    return y_hat


def predict_inner_test_fold(X, y, y_hat, train_idx, test_idx, alpha):
    """
    Predicts inner test fold.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    X_train, X_test = scale_features(X_train, X_test)
    y_hat[test_idx] = train_and_predict_ridge(alpha, X_train, y_train, X_test)
    return y_hat


def find_best_alpha(X, y, k_inner, alphas):
    """
    Finds the best alpha in an inner CV loop.
    """
    kf = KFold(n_splits=k_inner, shuffle=True)
    best_alpha = 0
    best_r2 = 0
    for idx, alpha in enumerate(alphas):
        y_hat = np.zeros_like(y)
        for train_idx, test_idx in kf.split(X):
            y_hat = predict_inner_test_fold(
                X, y, y_hat, train_idx, test_idx, alpha)
        r2 = stats.pearsonr(y, y_hat)[0] ** 2
        if r2 > best_r2:
            best_alpha = alpha
            best_r2 = r2
    print('best alpha', best_alpha)
    return best_alpha


def evaluate_fold(
    X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
        randomize):
    """
    Evaluates one fold of outer CV.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    if randomize:
        random.shuffle(y_train)
    best_alpha = find_best_alpha(X_train, y_train, k_inner, alphas)
    X_train, X_test = scale_features(X_train, X_test)
    y_test_hat = train_and_predict_ridge(best_alpha, X_train, y_train, X_test)
    r2 = stats.pearsonr(y_test, y_test_hat)[0] ** 2
    r2s[fold] = r2
    y_hat[test_idx] = y_test_hat
    return r2s, y_hat, fold + 1


In [9]:
_, _, _, r2 = predict_consumption(feats, y_log)
r2

best alpha 77.4263682681127
best alpha 77.4263682681127
best alpha 215.44346900318823
best alpha 10.0
best alpha 10.0


0.4019675446305446

In [10]:
_, _, _, r2 = predict_consumption(feats, y)
r2

best alpha 77.4263682681127
best alpha 599.4842503189409
best alpha 599.4842503189409
best alpha 599.4842503189409
best alpha 215.44346900318823


0.2718559522864684

In [33]:
y = clusters['cluster_hh_f34'].values

In [34]:
_, _, _, r2 = predict_consumption(feats, y)
r2

best alpha 77.4263682681127
best alpha 77.4263682681127
best alpha 27.825594022071243
best alpha 27.825594022071243
best alpha 77.4263682681127


0.5026909523726555

In [28]:
nas = ~clusters['cluster_hh_f35'].isna()

In [29]:
y = clusters['cluster_hh_f35'].values[nas] + 0.001
y_log = np.log(y)

In [30]:
_, _, _, r2 = predict_consumption(feats[nas], y_log)
r2

best alpha 215.44346900318823
best alpha 77.4263682681127
best alpha 77.4263682681127
best alpha 27.825594022071243
best alpha 215.44346900318823


0.25955890779287144

In [31]:
_, _, _, r2 = predict_consumption(feats[nas], y)
r2

best alpha 215.44346900318823
best alpha 77.4263682681127
best alpha 27.825594022071243
best alpha 215.44346900318823
best alpha 599.4842503189409


0.2060134595103929